In [ ]:
from pathlib import Path
import pandas as pd
import datetime as dt

# Tutorial for generating domestic hot water and grey water profiles 
This tutorial uses the DHWaterComsumption and GreyWaterConsumption method from <code>tsgenerator</code> in **CorrAI**. It aims at showing how to calculate the domestic hot and cold water consumption, using different methods.

# Domestic hot water
First, let us generate DHW consumption, either calculated with RE2020 or COSTIC coefficients.

### Load class
First, load the class <code>DHWaterConsumption</code> for timeseries generation

In [ ]:
from corrai.tsgenerator import DomesticWaterConsumption

Set the start and the end of your study

In [ ]:
start = dt.datetime(2023, 1, 1, 0, 0)
end   = dt.datetime(2024, 1, 1, 0, 0)

### Set parameters
Set various parameters, such as volume of water used for showers (L) and flow rates (L/min), time spent (min) for showers, etc.

In [ ]:
n_dwellings=15
v_per_dwelling=112
ratio_bath_shower=1
t_shower=7
d_shower=8
s_tot_building=481.4
s_moy_dwelling=60
n_people_per_dwelling=2

### RE2020 case
The formula taken into account in RT2020 for domestic hot water consumption calculates linear consumptions per hour spread over 5 hours. It is based on a product of given coefficients (dhw.coefficients), that differ from hour to hour, weekday to weekend, and month to month

Let's create an object specifying the calculation method to "RE2020". 
Mandatory argument is <code>n_dwellings</code>.

In [ ]:
dhw = DomesticWaterConsumption(
    n_dwellings=n_dwellings,
    v_per_dwelling=v_per_dwelling,
    ratio_bath_shower=ratio_bath_shower,
    t_shower=t_shower,
    d_shower=d_shower,
    s_tot_building=s_tot_building,
    s_moy_dwelling=s_moy_dwelling,
    n_people_per_dwelling=n_people_per_dwelling,
    method= "RE2020",
)

Now we can get the shower consumption distribution for the entire building (L/h) 

In [ ]:
# Calculation consumption ECS
df_re = dhw.re2020_shower_distribution(
    start = start, 
    end = end
)

If wanted, you can get for all timesteps the calculated coefficients based on <code>dhw.coefficients</code> with:

In [ ]:
# [optional to run] RE2020 coefficients
RE2020_coef = dhw.get_coefficient_calc_from_period(
        start = start, 
        end = end,
)
RE2020_coef

Plot the flow rate of hot water related to showers for a few days in may. 

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_re.loc["2023-05-01":"2023-05-07"].index,
    y=df_re['Q_ECS_RE2020'],
    fill=None,
    mode='lines',
    line_color='rgba(250,120,50,1)',
))

fig.update_layout(
    title='DHW consumtion using RE2020 calculation',
    xaxis_title='Time',
    yaxis_title='[L/h]')

fig.show()

In reality, the consumptions are much more spread out over the day.

### COSTIC Case

Using measurement data given in « Guide technique les besoins en eau chaude sanitaire », new coefficients were calculated by the COSTIC (https://www.costic.com/) to provide more realistic hourly DHW distribution.


In [ ]:
dhw = DomesticWaterConsumption(
    n_dwellings=n_dwellings,
    v_per_dwelling=v_per_dwelling,
    ratio_bath_shower=ratio_bath_shower,
    t_shower=t_shower,
    d_shower=d_shower,
    s_tot_building=s_tot_building,
    s_moy_dwelling=s_moy_dwelling,
    n_people_per_dwelling=n_people_per_dwelling,
    method= "COSTIC",
)

In [ ]:
# DHW consumption
df_costic = dhw.costic_shower_distribution(
        start = start, 
        end = end,
)
df_costic

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_costic.loc["2023-05-01":"2023-05-07"].index,
    y=df_costic['Q_ECS_COSTIC'],
    fill=None,
    mode='lines',
    line_color='gold',
#     name="DHW consumtion"
))

fig.update_layout(
    title='DHW consumtion using COSTIC coefficients',
    xaxis_title='Time',
    yaxis_title='[L/h]')

fig.show()

The consumption is hourly calculated. For realistic peaks in the DHW, you can create random distribution per minute using <code>costic_random_shower_distribution</code>.

Argument <code>optional_columns</code> (default=False) returns additional columns :
- number of showers
- cumulative time of showers/minutes


Option <code>seed</code> allows to have the same "random" distribution over again in case you want to regenerate the file with the same values.

In [ ]:
df_rand = dhw.costic_random_shower_distribution(
        start = start, 
        end = end,
        optional_columns = True, 
        seed = 42
)
df_rand

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_rand.loc["2023-05-01":"2023-05-07"].index,
    y=df_rand['Q_ECS_COSTIC_rd'],
    fill=None,
    mode='lines',
    line_color='darkblue',
#     name="DHW consumtion"
))

fig.update_layout(
    title='DHW consumtion using COSTIC coefficients with random distribution',
    xaxis_title='Time',
    yaxis_title='[L/h]')

fig.show()

### Plot all data
Let's compare all three methods.

In [ ]:
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(
    x=df_rand.loc["2023-05-01":"2023-05-03"].index,
    y=df_rand["Q_ECS_COSTIC_rd"],
    name = 'consoECS_COSTIC_random', 
    line_color='rgba(0,0,139,0.5)',
),secondary_y=True)

fig.add_trace(go.Scatter(
    x=df_re.loc["2023-05-01":"2023-05-03"].index,
    y=df_re['Q_ECS_RE2020'],
    name = 'RE2020', 
    fill = 'tozeroy',
    line_color='rgba(250,120,50,1)',
    fillcolor='rgba(250,120,50,0.8)',
),secondary_y=False)

fig.add_trace(go.Scatter(
    x=df_costic.loc["2023-05-01":"2023-05-03"].index,
    y=df_costic["Q_ECS_COSTIC"],
    name = 'consoECS_COSTIC', 
    line_color='gold',
),secondary_y=False)


fig.update_layout(
    title='DHW consumption (two axes)',
    xaxis_title='Date',
    yaxis_title='[L/h]',
)


## Grey waters from appliances
Now, let's get the grey waters for the same period from appliances such as the dish washer or the washing machine.

You can set parameters such as :
- Volume of water used for dish or washing machine in L (<code>v_water_dish</code> or <code>v_water_clothes</code>)
- Number of cycles per person per year (<code>cycles_clothes_pers</code> or <code>cycles_dish_pers</code>)
- Duration of cycles in hour (<code>duration_dish</code> or <code>duration_clothes</code>)
- <code>dish_washer</code>: Whether the household has a dishwasher. Defaults to True.
- <code>washing_machine</code>: Whether the household has washing machine. Defaults to True.


In [ ]:
appliances_waters = dhw.appliances_water_distribution(start, end, seed = 42)
appliances_waters

In [ ]:
# Create figure
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(
    x=appliances_waters.loc["2023-05-01":"2023-05-15"].index,
    y=appliances_waters["Q_dish"],
    name = 'Dish washer', 
    line_color='darkblue',
),secondary_y=False)

fig.add_trace(go.Scatter(
    x=appliances_waters.loc["2023-05-01":"2023-05-15"].index,
    y=appliances_waters["Q_washer"],
    name = 'Q_washing machine', 
    line_color='gold',
),secondary_y=False)

fig.update_layout(
    title='Grey water consumption',
    xaxis_title='Date',
    yaxis_title='[L/h]',
)


## Domestic cold waters
Finally, we can calculate the cold water usage, such as washbasin, cooking, sink dish, and sink cleaning. 

You can set percentages of usages of all cold water for :
- Showers with <code>percent_showers</code> (default to 40%) 
- Washbasin with <code>percent_washbasin</code> (default to 13%) 
- Cooking with <code>percent_cook</code> (default to 7%) 
- Dished with <code>percent_dishes</code> (default to 4%) 
- Cleaning with <code>percent_cleaning</code> (default to 6%) 

**CAREFUL, this code takes much longer than the previous ones (expect a few minutes for an entire year)**
Let's try for a few days in july.

In [ ]:
start2 = dt.datetime(2023, 7, 1, 0, 0)
end2 = dt.datetime(2023, 7, 20, 0, 0)

cold_waters = dhw.costic_random_cold_water_distribution(start2, end2, seed = 42)
cold_waters

# Concatenate all Data 

Resample hourly flow rates to new frequency (here 1 minute)

In [ ]:
from corrai.tsgenerator import resample_flow_rate

df = pd.DataFrame({
    "COSTIC_shower": df_costic["Q_ECS_COSTIC"],
    "RE2020_shower": df_re["Q_ECS_RE2020"],   
    "appliance_dish": appliances_waters["Q_dish"],
    "appliance_washer": appliances_waters["Q_washer"],
})

new_freq = '1T'
df_resampled = resample_flow_rate(df, new_freq)

Concatenate minute flow rates (random distributions)

In [ ]:
df_random = pd.DataFrame({
    "COSTIC_rd_shower": df_rand["Q_ECS_COSTIC_rd"],
    "COSTIC_rd_washbasin": cold_waters["Q_washbasin_COSTIC_rd"],
    "COSTIC_rd_cook": cold_waters["Q_sink_cook_COSTIC_rd"],
    "COSTIC_rd_dishes": cold_waters["Q_sink_dishes_COSTIC_rd"],
    "COSTIC_rd_cleaning": cold_waters["Q_sink_cleaning_COSTIC_rd"],
}) 

Concatenate all data

In [ ]:
df_all = pd.concat([df_resampled, df_random], axis = 1)
df_all = df_all.truncate(before=df_all.index[0], after=df_all.index[-1] - pd.Timedelta(days=1))

Plot all data for one day

In [ ]:
import plotly.express as px

start2 = dt.datetime(2023, 7, 16, 0, 0)
end2 = dt.datetime(2023, 7, 16, 23, 0)

df_jan = df_all.loc[start2:end2]
px.line(df_jan)

#### Powers based on delta temperatures
Now, let's estimate the required thermal power (in kW) to heat up the water. Function <code>calculate_power</code> takes as required arguments :
- a dataframe with flow rates in L/h 
- the targetted temperature delta (K)

Note that the function is applied to all data at once here (hence with a unique deltaT, while they actually depend on the water usage). For more accuracy, it would be best to apply the function to each usage, with specific deltas.

In [ ]:
from corrai.tsgenerator import calculate_power

# delta of temperature
deltaT_COSTIC = 55-13 
deltaT_RE2020 = 40-16
Cp_eau = 4186

# Create dataframe with hot waters only
powers = df_all[
    [
        "COSTIC_shower",
        "COSTIC_rd_shower",
        "RE2020_shower",
        "appliance_dish",
        "appliance_washer",
    ]
]

powers = calculate_power(powers, deltaT_COSTIC, Cp_eau)

Calculate the annual power consumption per person and usage (KWh)

In [ ]:
powers.sum()/n_dwellings/n_people_per_dwelling